# Base line In Sample Testing - Linear Regression

Test Type | Accuracy Score/Absolute Error
------------- | -------------
In Sample Testing 80/20 Split  | 91%
2024  | 2.94
2023 | 3.15
2022 | 3.32
2021 | 3.47
2020 | 3.49

Season |Start Date
------------- | -------------
In Sample Testing 80/20 Split  | 2014 - 2024
2024  | 2023-10-25
2023  | 2022-10-18
2022  | 2021-10-20
2021  | 2020-10-20
2020  | 2019-10-20

In [2]:
# Library Imports
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [4]:
#Loading our box score dataframe
df = pd.read_excel('../DataScience_Pipeline/RawDataFiles/combinedAdvancedRegularBoxScoreGamelogs.xlsx')

dfSingle = pd.read_excel('../DataScience_Pipeline/RawDataFiles/combinedAdvancedRegularBoxScoreGamelogs.xlsx')

In [93]:
# we need to combine the team 1 and team 2 fields into one common filed since we are going after total points as opposed to individual team points
df['Free_Throw_Attempts'] = df['FTA_Team']+df['FTA_Opponent']
df['Free_Throw_Make_Percentage'] = (df['FT_Perc_Team']+df['FT_Perc_Opponent'])/2
df['Total_3_Point_Goal_Attempts'] = df['ThreePA_Team']+df['ThreePA_Opponent']
df['Three_Point_Goal_Make_Percentage'] = df['ThreePA_Perc_Team']+df['ThreePA_Perc_Opponent']
df['Total_2_Point_Field_Goals_Made'] = df['FG_Team']+ df['FG_Opponent'] - (df['ThreeP_Team'] + df['ThreeP_Opponent'])
df['Total_2_Point_Goal_Attmepts'] = (df['FGA_Team']+df['FGA_Opponent'])
df['Two_Point_Goal_Make_Percentage'] = df['Total_2_Point_Field_Goals_Made']/df['Total_2_Point_Goal_Attmepts']
df['Offensive_EFG'] = (df['Team1_Offensive_EFG']+df['Team2_Offensive_EFG'])/2
df['Offensive_TurnOver_Rate'] = (df['Team1_Offensive_TurnOver_Rate'] + df['Team2_Offensive_TurnOver_Rate'])/2
df['Offensive_Rebounding_Rate'] = (df['Team1_Offensive_Rebounding_Rate'] + df['Team2_Offensive_Rebounding_Rate'])/2
df['Defensive_Rebounding_Rate'] = (df['Team1_Defensive_Rebounding_Rate'] + df['Team2_Defensive_Rebounding_Rate'])/2
df['Offensive_Rating'] = (df['Team1_OffensiveRating'] + df['Team2_OffensiveRating'])/2
df['Free_Throw_Attempt_Rate'] = (df['Team1_FreeThrowAttemptRate'] + df['Team2_FreeThrowAttemptRate'])/2
df['Three_Point_Attempt_Rate'] = (df['Team1_3PointAttemptRate'] + df['Team2_3PointAttemptRate'])/2

### Combined Variables - Dataframe Munging

In [94]:
# Subtracting fields from our dataframe that we dont need
df = df[[
        'Date',
        'Team1',
        'Opponent',
        'Points_Scored',
        'Free_Throw_Attempts',
        'Total_3_Point_Goal_Attempts',
        'GamePace',
        'Offensive_EFG',
        'Offensive_Rating'
         
        ]]

In [95]:
predictors = [
        'Free_Throw_Attempts',
        'Total_3_Point_Goal_Attempts',
        'GamePace',
        'Offensive_EFG',
        'Offensive_Rating'    
]

### In Sample Testing - Entire Data Set (Combined Variables)

In [1]:
X_train, X_test, y_train, y_test = train_test_split(df[predictors], df['Points_Scored'], test_size=0.30)
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)

NameError: name 'train_test_split' is not defined

In [2]:
lr.score(X_test,y_test)

NameError: name 'lr' is not defined

### In Sample Testing - Seasonal Basis (Combined Variables)

In [98]:
# Creating our Training data set
# THis data contains data points for the season prior to the 2024 season
dfTraining = df[df['Date']<'2023-10-26']
dfTraining = dfTraining[dfTraining['Date']>='2022-10-26'] # This line is not needed, just an additional way to limit the training data
# Creating our Testing data which is the 2024 season data
dfTesting = df[df['Date']>='2023-10-26']

In [99]:
#Creating our Test train Split on the data prior to the testing season
#Applying the linear regression to the model
X_train, X_test, y_train, y_test = train_test_split(dfTraining[predictors], dfTraining['Points_Scored'])
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [100]:
# Creating our predictions
y_pred=lr.predict(dfTesting[predictors])

In [101]:
# Creting a dataframe column for our predictions
dfTesting['Predictions'] = y_pred
# Determining the error between the points scored and the predictions
dfTesting['AbsoluteError'] = abs(dfTesting['Points_Scored'] - dfTesting['Predictions'])

In [102]:
# Finding the mean absolute error
dfTesting['AbsoluteError'].mean()

2.924334489965767

### Single Team Features - Dataframe Munging

In [46]:
#Loading our team level regular box score dataframe
regularBoxScoreDF = pd.read_excel('../DataScience_Pipeline/RawDataFiles/teamRegularBoxScoreGamelogs.xlsx')
#Loading our team level advanced box score dataframe
advancedBoxScoreDF = pd.read_excel('../DataScience_Pipeline/RawDataFiles/teamAdvancedBoxScoreGamelogs.xlsx')

In [47]:
advancedBoxScoreDF.columns

Index(['Unnamed: 0', 'Game', 'Date', 'Location', 'Team', 'Opponent', 'Result',
       'Team_Score', 'Opponent_Score', 'Offensive_EFG', 'Defensive_EFG',
       'Offensive_TurnOver_Rate', 'Defensive_TurnOver_Rate',
       'Offensive_Rebounding_Rate', 'Defensive_Rebounding_Rate',
       'OffensiveRating', 'DefensiveRating', 'Pace', 'FreeThrowAttemptRate',
       'ThreePointAttemptRate', 'TrueShootingRate'],
      dtype='object')

In [59]:
# Subtracting fields from our regular box score dataframe that we dont need
regularBoxScoreDF = regularBoxScoreDF[[
    'Date',
    'Team',
    #'FTA',
   # 'ThreePA',
    'Team_Points'
    
]]
# Subtracting fields from our advanced box score dataframe that we dont need
advancedBoxScoreDF = advancedBoxScoreDF[[
    'Date',
    'Team',
    'Pace',
    'Offensive_EFG',
    'OffensiveRating',
    'TrueShootingRate'
    
]]

# Team 1 DataFrame Join
Merged_df = pd.merge(regularBoxScoreDF, advancedBoxScoreDF, on=['Date','Team'], how='left')

In [60]:
# For this section of the testing, we will only look at Team1 as a sample
Merged_df = Merged_df[[
        'Date',
        'Team',
        'Team_Points',
        #'FTA',
        #'ThreePA',
        'Pace',
        'Offensive_EFG',
        'OffensiveRating',
        'TrueShootingRate'
         
        ]]

In [63]:
predictorsSingle = [
       # 'FTA',
       # 'ThreePA',
        'Pace',
        'Offensive_EFG',
        'OffensiveRating',
        'TrueShootingRate'    
]

### In Sample Testing - Entire Dataset (Single Team Features)

In [64]:
X_train, X_test, y_train, y_test = train_test_split(Merged_df[predictorsSingle], Merged_df['Team_Points'], test_size=0.30)
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)

In [65]:
lr.score(X_test,y_test)

0.9420645495098171

### In Sample Testing - Seasonal Basis (Single Team Features)

In [66]:
# Creating our Training data set
# THis data contains data points for the season prior to the 2024 season
dfTraining = Merged_df[Merged_df['Date']<'2023-10-26']
# Creating our Testing data which is the 2024 season data
dfTesting = Merged_df[Merged_df['Date']>='2023-10-26']

In [67]:
#Creating our Test train Split on the data prior to the testing season
#Applying the linear regression to the model
X_train, X_test, y_train, y_test = train_test_split(dfTraining[predictorsSingle], dfTraining['Team_Points'])
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [68]:
# Creating our predictions
y_pred=lr.predict(dfTesting[predictorsSingle])

In [69]:
# Creting a dataframe column for our predictions
dfTesting['Predictions'] = y_pred
# Determining the error between the points scored and the predictions
dfTesting['AbsoluteError'] = abs(dfTesting['Team_Points'] - dfTesting['Predictions'])

In [70]:
# Finding the mean absolute error
dfTesting['AbsoluteError'].mean()

1.3952411127271247

In [71]:
dfTesting

,Date,Team,Team_Points,Pace,Offensive_EFG,OffensiveRating,TrueShootingRate,Predictions,AbsoluteError
719,2023-10-26,MIL,118,101.7,0.574,116.1,0.609,118.356827,0.356827
720,2023-10-29,MIL,110,103.2,0.547,106.6,0.580,110.721045,0.721045
721,2023-10-30,MIL,122,101.5,0.575,120.2,0.609,122.159731,0.159731
722,2023-11-01,MIL,111,95.6,0.546,116.1,0.599,111.845716,0.845716
723,2023-11-03,MIL,110,97.9,0.549,112.3,0.583,110.582258,0.582258
...,...,...,...,...,...,...,...,...,...
23953,2024-04-07,GSW,118,94.1,0.609,125.4,0.623,119.170123,1.170123
23954,2024-04-09,GSW,134,100.1,0.750,133.8,0.751,133.547418,0.452582
23955,2024-04-11,GSW,100,90.9,0.545,110.0,0.583,100.775165,0.775165
23956,2024-04-12,GSW,109,98.5,0.541,110.6,0.572,109.579790,0.579790
